<a href="https://colab.research.google.com/github/RTVIENNA/1450-RAG-Preprocessing/blob/main/God_of_ducks_and_lamas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The 1450 Navigator 🧭

🕵🏻 Agentic RAG with 🦙 Llama 3.2 3B

The running ducks and the smart LAMA

🎯 Our goal is to create a system that answers questions using a knowledge base focused on medical information. If the retrieved documents don't contain the answer, the application will fall back to web search for additional context.

Stack:

🦆📄 Docling: Docling simplifies document processing, parsing diverse formats — including advanced PDF understanding — and providing seamless integrations with the gen AI ecosystem.
Parsing of multiple document formats incl. PDF, DOCX, XLSX, HTML, images, and more

🏗️ Haystack: open-source LLM orchestration framework that streamlines the development of your LLM applications.

🦙 Llama-3.2-3B-Instruct: small and good Language Model.

🦆🌐 DuckDuckGo API Websearch to search results on the Web.

Make sure that the T4 GPU is running

In [1]:
! pip install docling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.2/160.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.1/541.1 kB 36.2 MB/s eta 0:00:00
   

In [2]:
! pip install haystack-ai duckduckgo-api-haystack transformers sentence-transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.6/451.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 53.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.9
    Uninstalling dill-0.3.9:
      Successfully 

In [3]:
import os #3
from transformers import AutoModelForSequenceClassification #3

import gdown #4
from docling.document_converter import DocumentConverter #4
from haystack import Document #4
from haystack.document_stores.in_memory import InMemoryDocumentStore #4
from haystack.components.embedders import SentenceTransformersDocumentEmbedder #4

import torch #6
from haystack.components.generators import HuggingFaceLocalGenerator #6

from haystack.components.embedders import SentenceTransformersTextEmbedder #8
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever #8

from haystack.components.builders import PromptBuilder #9

from haystack.components.routers import ConditionalRouter #10

from duckduckgo_api_haystack import DuckduckgoApiWebSearch #11

from haystack.components.joiners import BranchJoiner #14
prompt_joiner  = BranchJoiner(str)

from haystack import Pipeline #14



Access to external Documents that are saved in a google drive. The data is cleaned/ chuncked and embedded

In [5]:
# 1. Download files from Google Drive
url = "https://drive.google.com/drive/u/0/folders/1YrBIqbbi5uXjR-fuEAMBHL-TwpjtViXu"
output_dir = "1450_files"
gdown.download_folder(url, quiet=True, output=output_dir)

# 2. Process each downloaded file
import os
for filename in os.listdir(output_dir):
    if filename.endswith(".pdf"):  # Adjust file extension if needed
        filepath = os.path.join(output_dir, filename)

        # 3. Convert the PDF file
        converter = DocumentConverter()
        result = converter.convert(filepath)
        converted_text = result.document.export_to_markdown()
        print(result.document.export_to_markdown())

        # 4. Create a Haystack Document object
        doc = Document(content=converted_text, meta={"source": filepath})

<!-- image -->

Notaufnahmen Campus Charité Mitte und Campus Virchow-Klinikum | CCM | CVK

Startseite  Notfallversorgung  Manchester Triage System

## Manchester Triage System

Triage bedeutet Ersteinschätzung.

Eine medizinische Fachkraft beurteilt nach einem standardisierten Vorgehen die Dringlichkeit der Behandlung.

<!-- image -->

## Das Manchester Triage System Ersteinschätzung der Behandlungsdringlichkeit

Das Manchester Triage System (MTS) ist ein standardisiertes Verfahren zur systematischen Ersteinschätzung bzw. Triage der Behandlungsdringlichkeit von Patienten in Rettungs-stellen bzw. Notaufnahmen. Triage bezeichnet dabei die Methodik, den Schweregrad der Erkrankung bzw. der Verletzung innerhalb kurzer Zeit zu erkennen und mittels Kategorisierung eine Einstufung der Behandlungsdringlichkeit vorzunehmen. Die schnelle und sichere Festlegung der Behandlungspriorität ist eine Grundvoraussetzung, um bei begrenzten Ressourcen eine dringliche notwendige medizinische Behandlung gg

In [6]:
# 3. Einrichten des In-Memory-Dokumentenspeichers
document_store = InMemoryDocumentStore()

# 4. Initialisieren und Aufwärmen des Document Embedder
doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
doc_embedder.warm_up()

# 5. Embedding des Dokuments und Schreiben in den Dokumentenspeicher
docs_with_embeddings = doc_embedder.run([doc])
document_store.write_documents(docs_with_embeddings["documents"])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

1

Authorization

you need an Hugging Face account-> Create an access token and set it to read

In [7]:
import getpass, os

os.environ["HF_TOKEN"] = getpass.getpass("Your Hugging Face token")

Your Hugging Face token··········


In [8]:
generator = HuggingFaceLocalGenerator(
    model="meta-llama/Llama-3.2-3B-Instruct",
    huggingface_pipeline_kwargs={"device_map":"auto",
                                 "torch_dtype":torch.bfloat16},
    generation_kwargs={"max_new_tokens": 256})

generator.warm_up()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [9]:
prompt = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>
  What is the manchester Triage Algorithm?<|eot_id|>
  <|start_header_id|>assistant<|end_header_id|>"""

generator.run(prompt)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'replies': ["\n\nThe Manchester Triage Algorithm is a method used to prioritize patients in emergency departments (EDs) based on the severity of their condition. It was developed by the Manchester Royal Infirmary in the UK in the 1970s.\n\nThe algorithm is a simple, visual tool that categorizes patients into four color-coded groups based on their clinical urgency and acuity. The colors used are:\n\n1. **Red**: Patients who require immediate attention, often with life-threatening conditions such as severe trauma, cardiac arrest, or severe respiratory distress.\n2. **Yellow**: Patients who require urgent attention, but are not life-threatening, such as those with injuries that require surgical intervention or severe illnesses that require close monitoring.\n3. **Green**: Patients who can be managed in a less urgent setting, such as those with minor injuries or illnesses that can be treated at an outpatient clinic.\n4. **Black**: Patients who are deceased or have been pronounced dead.\n\

Build the 🕵🏻 Agentic RAG Pipeline


Retrieval part- initialize the components to use for the initial retrieval phase.

In [10]:
text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
retriever = InMemoryEmbeddingRetriever(document_store, top_k=5)

Prompt template
Let's define the first prompt template, which instructs the model to:

answer the query based on the retrieved documents, if possible
reply with 'no_answer', otherwise

In [11]:
prompt_template = """
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Answer the following query given the documents.
If the answer is not contained within the documents reply with 'no_answer'.
If the answer is contained within the documents, start the answer with "FROM THE KNOWLEDGE BASE: ".

Documents:
{% for document in documents %}
  {{document.content}}
{% endfor %}

Query: {{query}}<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
"""

prompt_builder = PromptBuilder(template=prompt_template)

Conditional Router
This is the component that will perform data routing, depending on the reply given by the Language Model.

In [12]:
routes = [
    {
        "condition": "{{'no_answer' in replies[0]}}",
        "output": "{{query}}",
        "output_name": "go_to_websearch",
        "output_type": str,
    },
    {
        "condition": "{{'no_answer' not in replies[0]}}",
        "output": "{{replies[0]}}",
        "output_name": "answer",
        "output_type": str,
    },
]

router = ConditionalRouter(routes)

In [13]:
router.run(replies=["this is the answer!"])

{'answer': 'this is the answer!'}

In [14]:
router.run(replies=["no_answer"], query="my query")

{'go_to_websearch': 'my query'}

WEB search with duckduckgo

In [15]:
websearch = DuckduckgoApiWebSearch(top_k=5)

In [16]:
# Perform a search
results = websearch.run(query="Triage Algortihm?")

# Access the search results
documents = results["documents"]
links = results["links"]

print("Found documents:")
for doc in documents:
    print(f"Content: {doc.content}")

print("\nSearch Links:")
for link in links:
    print(link)

/usr/local/lib/python3.11/dist-packages/duckduckgo_api_haystack/duckduckgoapi.py:176: UserWarning: backend='api' is deprecated, using backend='auto'
  results = self.ddgs.text(**payload)


Found documents:
Content: Background information START was developed by the Newport Beach Fire and Marine Department and Hoag Hospital in Newport Beach, California in 1983. 2 Initially it used the ability to obey commands, respiratory rate, and capillary refill to assign triage category. Modifications to START in 1996 by Benson et. al. substituted radial pulse for capillary refill, with a report of improved accuracy ...
Content: In the hospital triage systems, the ESI triage model amongst five-level triage systems, which has a higher level of validity and reliability, and the CRAMS triage system used to triage patients in the emergency units of the hospitals were identified. These systems are described according to the following algorithm. START triage system
Content: Triage is the process used to assess patients' injuries or illnesses and determine the priority of care. Different levels of triage indicate who should get emergency medical attention first. Learn more about the different

Prompt template after Web search

In [17]:
prompt_template_after_websearch = """
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Answer the following query given the documents retrieved from the web.
Start the answer with "FROM THE WEB: ".

Documents:
{% for document in documents %}
  {{document.content}}
{% endfor %}

Query: {{query}}<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
"""

prompt_builder_after_websearch = PromptBuilder(template=prompt_template_after_websearch)

Assembling the Pipeline

In [18]:
prompt_joiner  = BranchJoiner(str)

In [19]:
pipe = Pipeline()
pipe.add_component("text_embedder", text_embedder)
pipe.add_component("retriever", retriever)
pipe.add_component("prompt_builder", prompt_builder)
pipe.add_component("prompt_joiner", prompt_joiner)
pipe.add_component("llm", generator)
pipe.add_component("router", router)
pipe.add_component("websearch", websearch)
pipe.add_component("prompt_builder_after_websearch", prompt_builder_after_websearch)

pipe.connect("text_embedder", "retriever")
pipe.connect("retriever", "prompt_builder.documents")
pipe.connect("prompt_builder", "prompt_joiner")
pipe.connect("prompt_joiner", "llm")
pipe.connect("llm.replies", "router.replies")
pipe.connect("router.go_to_websearch", "websearch.query")
pipe.connect("router.go_to_websearch", "prompt_builder_after_websearch.query")
pipe.connect("websearch.documents", "prompt_builder_after_websearch.documents")
pipe.connect("prompt_builder_after_websearch", "prompt_joiner")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - prompt_joiner: BranchJoiner
  - llm: HuggingFaceLocalGenerator
  - router: ConditionalRouter
  - websearch: DuckduckgoApiWebSearch
  - prompt_builder_after_websearch: PromptBuilder
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> prompt_joiner.value (str)
  - prompt_joiner.value -> llm.prompt (str)
  - llm.replies -> router.replies (List[str])
  - router.go_to_websearch -> websearch.query (str)
  - router.go_to_websearch -> prompt_builder_after_websearch.query (str)
  - websearch.documents -> prompt_builder_after_websearch.documents (List[Document])
  - prompt_builder_after_websearch.prompt -> prompt_joiner.value (str)

Agentic RAG in action! 🔎

In [20]:
def get_answer(query):
  result = pipe.run({"text_embedder": {"text": query}, "prompt_builder": {"query": query}, "router": {"query": query}})
  print(result["router"]["answer"])

In [21]:
query = "What is the Manchester Triage Algorithm ?"

get_answer(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




FROM THE KNOWLEDGE BASE: The Manchester Triage Algorithm is a system used to assess the priority of patients in an emergency department or intensive care unit. It is a symptom-oriented system that does not rely on diagnoses, but instead focuses on the patient's current symptoms to determine the level of urgency. The algorithm assigns patients to one of five levels of priority, with the highest priority being "Sofort, vital" (immediate attention required) and the lowest being "nicht dringend" (not urgent). The algorithm also includes a " Ziel-Zeit" (target time) for each level of priority, which is the maximum time a patient can wait before seeing a doctor. The algorithm is used to ensure that patients receive appropriate care in a timely manner and to prevent unnecessary delays or overloads in the emergency department.


In [22]:
query = "I have fever, what should I do ?"

get_answer(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/usr/local/lib/python3.11/dist-packages/duckduckgo_api_haystack/duckduckgoapi.py:176: UserWarning: backend='api' is deprecated, using backend='auto'
  results = self.ddgs.text(**payload)
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




FROM THE WEB: If you have a fever, it's generally recommended to avoid caffeine, sugary soda, energy drinks, and sports drinks, as they can cause nausea and vomiting. It's also best to avoid cold baths or showers. However, if your fever is high (over 104°F or 40°C) or accompanied by certain symptoms such as a seizure, loss of consciousness, confusion, stiff neck, trouble breathing, severe pain, or swelling/inflammation, you should seek medical help right away. If your fever is low-grade (around 99.5-100.3°F), it's best to let it run its course, but try to stay hydrated and avoid alcohol rubdowns, which can do nothing to reduce fever and may cause other issues.


In [23]:
query = "I have chest pain?"

get_answer(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




FROM THE KNOWLEDGE BASE: 

No, the answer is not explicitly stated in the documents.
